In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *
from fastai.callbacks import *

In [7]:
torch.cuda.is_available()

True

True

In [4]:
np.random.seed(42)

In [ ]:
tfms = get_transforms(do_flip=True,flip_vert=False,max_rotate=10.0,max_zoom=1.2,max_lighting=0.5,max_warp=0.2,p_affine=0.75,p_lighting=0.75)


In [ ]:
#loss_func = LabelSmoothingCrossEntropy()

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1., gamma=1.):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets, **kwargs):
        CE_loss = nn.CrossEntropyLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-CE_loss)
        F_loss = self.alpha * ((1-pt)**self.gamma) * CE_loss
        return F_loss.mean()
    
focal_loss_func = FocalLoss(gamma=3.0)

In [ ]:
alpha=0.99
mom=0.9
eps=1e-6

In [ ]:
opt_func = partial(optim.Adam, betas=(mom,alpha), eps=eps)

In [ ]:
src = (ImageList.from_folder(path='train',include=["class-811","class-394","class-526","class-4","class-536","class-204"]).split_by_rand_pct(0.2).label_from_folder())

In [ ]:
data = src.transform(tfms,size=(512)).databunch(bs=20).normalize(imagenet_stats)

In [ ]:
learn = cnn_learner(data, models.densenet201,pretrained=True, metrics=[error_rate, accuracy, FBeta(beta=1,average='macro')],loss_func = focal_loss_func,opt_func=opt_func,wd=1e-2)

In [ ]:
data.show_batch(rows=3, figsize=(12,12))

In [ ]:
learn = learn.to_fp16(dynamic=True)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
lr=5e-3

In [ ]:
learn.fit_one_cycle(10,lr)

In [ ]:
learn.save('thamnophis-round-1')

In [ ]:
lrs=[1e-6,1e-5,5e-4]

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(40,lrs)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)


In [ ]:
losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
interp.plot_confusion_matrix(figsize=(20,20), dpi=75)


In [ ]:
interp.plot_confusion_matrix(figsize=(20,20), dpi=75,normalize=True)


In [ ]:
interp.most_confused(min_val=2)


In [ ]:
learn.unfreeze()

In [ ]:
learn.save('thamnophis-round-2')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)


In [ ]:
interp.plot_confusion_matrix(figsize=(20,20), dpi=75,normalize=True)


In [ ]:
interp.plot_confusion_matrix(figsize=(20,20), dpi=75,normalize=False)

In [ ]:
interp.plot_top_losses(6, figsize=(30,22))

In [ ]:
learn.loss_func = focal_loss_func

In [ ]:
learn